Stella MIRRI,  Jack EVANS
# PROJET 1 :   Rangement d’un tuyau de jardin

 On modélise la position du tuyau comme une courbe de longueur L dans le plan. On représente cette
 courbe par $N+1$ points $$(xi, yi) \qquad (i = 0, ... N)$$ uniformément espacés d’une longueur $ds = \frac{L}{N}$. 
 
 En introduisant l’angle $\theta_i$ de la courbe avec l’origine au point $(xi,\ yi)$, on obtient
 $$x_{i+1}- xi =ds \ cos \theta_i$$
 $$y_{i+1}- yi =ds \ sin \theta_i$$
 $$\qquad (i=0,... N) \qquad(1)$$
 où les variations de $\theta_i$ sont limitées, du fait de la courbure maximale $\alpha > 0$ du tuyau, via la condition
 $$\theta_{i+1}-\theta_i \in [ - \alpha ds, \alpha ds] \qquad (2)$$

 On suppose que le robinet d’eau est situé au point $(0,\ 1)$ tandis que le parterre de fleur doit être arrosé
 au point $(\frac{L}{2},\ 0)$ et l’on impose
 $$x_0 = \theta_0 =y_N = \theta_{N-1}=0 \qquad y_0=1 \qquad (3)$$
 On cherche alors à minimiser la fonction
$$K(x_N-\frac{L}{2})^2 + ds\sum_{i=0}^N{|y_i|} \qquad (4)$$
 où $K >>ds$ est un paramètre constant.

## 1) Etude du problème d'optimisation


1. Interpréter le cout $(4)$. Que cherche-t-on a minimiser?

***
**REPONSE**

On cherche a minimiser : 
- d'une part   $(x_N-\frac{L}{2})^2$   pour faire avoir   $x_N=\frac{L}{2}$   ou plutôt avoir une valeur *suffisamment proche* pour faire l'approximation en pratique (d'où le $K$ grand pour donner plus de poids a ce terme).
- d'autre part   $ds\sum_{i=0}^N{|y_i|}=\frac{L}{N}\sum_{i=0}^N{|y_i|}$   pour que le tuyau soit le "plus proche possible" de l'axe des abscisses $y=0$ (faire 2 dessins)


***

2. Pourquoi n’a-t-on pas plutôt imposé la contrainte $x_N = \frac{L}{2}$ ? (Indice : on pourra considérer la valeur initiale qu’il faudrait alors fournir au solveur pour la résolution numérique d’un tel problème.)

***
**REPONSE**


***

3. Quelle difficulté peut présenter le coût $(4)$ numériquement ?

***
**REPONSE**


***

4. Justifier que minimiser $(4)$ est équivalent à minimiser
$$K(x_N − \frac{L}{2})^2+ ds\sum{wi}$$
t.q. $wi ≥ yi$ et $wi ≥ −yi , i = 0, . . . , N (5)$
où $wi (i = 0, . . . , N )$ sont de nouvelles variables scalaires. Quel est l’avantage de cette réécriture ? On la retiendra pour la suite du sujet.

***
**REPONSE**


***

5. Formuler le problème d’optimisation à résoudre sous la forme
$$min_{(c_{eq}(z)=0 , \ c_{ineq}(z)≤0)} f(z) \qquad (6)$$
On précisera les variables de décision $z$, leur nombre $n$, les contraintes $ceq$ et $cineq$ ainsi que la fonction objectif $f$ à minimiser

***
**REPONSE**


***

## 2) Etude et résolution numériques